## 1. Using `BaseModel` (Pydantic)

### What this method is

You define a Pydantic model (`Country(BaseModel)`) and pass the class to `llm.with_structured_output(Country)`.
LangChain uses the model’s type hints + field metadata to:

- Build a JSON schema behind the scenes
- Validate / parse the LLM response into a **real Python object** of type `Country`

### Things to remember

- **Strong validation**:
  - Types are enforced (`str`, `int`, `float`, etc.)
  - Missing / extra fields are checked and can raise validation errors.
- **Best when you want Python objects as the final result**:
  - e.g. `country.name`, `country.language`, `country.capital` with autocomplete in your IDE.
- **Good for bigger / reusable schemas**:
  - You can reuse the Pydantic model anywhere else in your project (DB models, API schemas, etc.).
- You can add defaults, regex, `Field(..., description="...")`, etc. to control both docs and validation.
- Slightly **heavier dependency**: relies on Pydantic, but that’s standard in many Python projects.

### When to use

- You want **strict structure + validation**.
- You want **Pythonic, IDE-friendly** objects.
- You already use Pydantic in FastAPI / other parts of your code.


In [17]:
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-flash-lite-latest")

class Country(BaseModel):

    """Information about a country"""

    name: str = Field(description="name of the country")
    language: str = Field(description="language of the country")
    capital: str = Field(description="Capital of the country")

structured_llm = llm.with_structured_output(Country)
structured_llm

RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-flash-lite-latest', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002180D702690>, default_metadata=()), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Country', 'description': 'Information about a country', 'parameters': {'properties': {'name': {'description': 'name of the country', 'type': 'string'}, 'language': {'description': 'language of the country', 'type': 'string'}, 'capital': {'description': 'Capital of the country', 'type': 'string'}}, 'required': ['name', 'language', 'capital'], 'type': 'object'}}}]}, config={}, config_factories=[])
| PydanticToolsParser(first_tool_only=True, tools=[<class '__main__.Country'>])

In [3]:
structured_llm.invoke("Tell me about Sri Lanka")

Country(name='Sri Lanka', language='Sinhala', capital='Sri Jayawardenepura Kotte')

## 2. Using `TypedDict` + `Annotated`

### What this method is

You create a `TypedDict` (`class Joke(TypedDict)`) and use `Annotated` to attach descriptions to each key.
Passing `Joke` into `llm.with_structured_output(Joke)` lets LangChain:

- Generate a JSON schema from the `TypedDict` + `Annotated` info
- Return a **plain Python `dict`** that matches your type hints

### Things to remember

- **Lighter-weight than Pydantic**:
  - No Pydantic dependency, just standard typing tools.
- Still gives **type information**:
  - Great for static type checkers (mypy, pyright) and IDE autocomplete.
- `Annotated[type, default, "description"]` is interpreted as:
  - The base type (`str`, `int`, etc.)
  - An optional default (often `...` is used as “no default”)
  - A human-readable description for schema / docs.
- **No runtime validation like Pydantic**:
  - It’s mostly for schema generation + editor help, not strict validation.

### When to use

- You want **lightweight structured output** with minimal dependencies.
- You’re okay with **dicts instead of full model objects**.
- You care about **type hints + nice autocomplete**, but heavy validation is not critical.


In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-flash-lite-latest")

from typing_extensions import Annotated, TypedDict
from typing import Optional


# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str,..., "The setup of the joke"]

    # Alternatively, we could have specified setup as:

    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description

    punchline: Annotated[str,..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None,"How funny the joke is, from 1 to 10"]


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

[{'args': {'punchline': 'Because they always land on their feet!',
   'setup': 'Why are cats so good at video games?'},
  'type': 'Joke'}]

## 3. Using a raw JSON Schema dict

### What this method is

You manually write a JSON schema dictionary (with `type`, `properties`, `required`, etc.) and pass it to:
```python
structured_llm = llm.with_structured_output(json_schema)


In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-flash-lite-latest")

json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}
structured_llm = llm.with_structured_output(json_schema)

structured_llm.invoke("Tell me a joke about cats")

[{'args': {'punchline': 'He was a purr-fessional.',
   'setup': 'Why did the cat get a job at the bank?'},
  'type': 'joke'}]